In [1]:
import pandas as pd
import random
import os
import ast

# Master Movie Data Manipulation Notebook

How this notebook could work is that we iterate/clean on the 'meta' csv in pandas, and then save it in the repo after we have added the new/corrected/cleaned data

In [2]:
#### change pathname to your clean dataset
meta = pd.read_csv('/Users/nickparker/Documents/USF_MSDS/Module_1/593-Visualization/final_project/the-movies-dataset-clean/movies_metadata.csv')
#meta.head(2).T

/Users/nickparker/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(meta.columns)

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


In [4]:
### Adding profit and use of Budget Columns
meta['profit'] = meta['revenue'] - pd.to_numeric(meta['budget'],errors='coerce')
meta['useOfBudget'] = meta['revenue'] / (pd.to_numeric(meta['budget'],errors='coerce') + 000.1)

In [5]:
##Adding production company
#Added Production Companies as a trend
productionCompanies1 = []
for rowNum in range(meta.shape[0]):
    try: 
        productionCompanies1.append(ast.literal_eval(meta['production_companies'][rowNum])[0]['name'])
    except:
        productionCompanies1.append('nan')
        
productionCompanies2 = []        
for rowNum in range(meta.shape[0]):
    try: 
        productionCompanies2.append(ast.literal_eval(meta['production_companies'][rowNum])[1]['name'])
    except:
        productionCompanies2.append('nan')
        
meta['ProductionCo1'] = productionCompanies1
meta['ProductionCo2'] = productionCompanies2

In [11]:
##Adding genre
genre1 = []
for rowNum in range(meta.shape[0]):
    try: 
        genre1.append(ast.literal_eval(meta['genres'][rowNum])[0]['name'])
    except:
        genre1.append('nan')
        
genre2 = []        
for rowNum in range(meta.shape[0]):
    try: 
        genre2.append(ast.literal_eval(meta['genres'][rowNum])[1]['name'])
    except:
        genre2.append('nan')

genre3 = []        
for rowNum in range(meta.shape[0]):
    try: 
        genre3.append(ast.literal_eval(meta['genres'][rowNum])[2]['name'])
    except:
        genre3.append('nan')
meta['genre1'] = genre1
meta['genre2'] = genre2
meta['genre3'] = genre3

## Merging

In [7]:
## Don't run this cell unless you want to merge in ratings (big file)
ratings = pd.read_csv('/Users/nickparker/Documents/USF_MSDS/Module_1/593-Visualization/final_project/the-movies-dataset-clean/ratings.csv')
print(ratings.shape)
links = pd.read_csv('/Users/nickparker/Documents/USF_MSDS/Module_1/593-Visualization/final_project/the-movies-dataset-clean/links.csv')
print(links.shape)

(26024289, 4)
(45843, 3)


In [8]:
### get average rating
ratings = ratings.groupby('movieId').agg({'rating':['mean']}).reset_index()
ratings.columns = ['movieId','rating']
print(ratings.shape)
linkAvgRating = pd.merge(links,ratings,on='movieId',how='left')
meta['imdbId'] = pd.to_numeric(meta['imdb_id'].str[2:])
meta = pd.merge(meta,linkAvgRating,on='imdbId',how='left')

(45115, 2)


In [9]:
print(links.imdbId.max()) #This is the imdb ID without the tt
print(links.movieId.max()) #This is the internal ID used in rankings

7158814
176279


(45843, 4)


In [12]:
meta.head(2).T

,0,1
adult,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN
budget,30000000,65000000
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
homepage,http://toystory.disney.com/toy-story,NaN
id,862,8844
imdb_id,tt0114709,tt0113497
original_language,en,en
original_title,Toy Story,Jumanji
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...


# Export

#### When you are done with the data manipulation, save as csv in our manipulated data folder

In [19]:
export_csv = meta.to_csv (r'movie_dataset_edited/meta.csv', header=True)
#Don't forget to add '.csv' at the end of the path